In [ ]:
import numpy as np
import glob
from nptdms import TdmsFile
import idas_preprocessing as idas
import matplotlib.pyplot as plt
import argparse
from functools import partial
import os
import multiprocessing

def das_read(filename):
    data, _, dt, dx = idas.read_tdms(filename)
    return data, dt, dx

def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--path',type=str)
    opt = parser.parse_args()
    return opt
data_list = np.loadtxt('sc_data.txt')
datapath = './data'
import utm
loc = np.load('xfj_ch.loc.npy')

fiber_loc = np.load('xfj_ch.loc.npy')
fiber_loc = utm.from_latlon(fiber_loc[:,1],fiber_loc[:,2])
fiber_loc = np.array(fiber_loc[0:2])
fiber_loc[0] = fiber_loc[0]-min(fiber_loc[0])
fiber_loc[1] = fiber_loc[1]-min(fiber_loc[1])
plt.figure(figsize=(15,10))
plt.rcParams.update({'font.size': 20})
plt.scatter(fiber_loc[0][1000-235:1300-235],fiber_loc[1][1000-235:1300-235],label='Fiber',s=10)
plt.scatter(fiber_loc[0][1000-235:1300-235:5],fiber_loc[1][1000-235:1300-235:5],label='Fiber',s=100)
plt.scatter(205,40,s=100,c='r')
plt.xlabel(xlabel='x (m)')
plt.ylabel(ylabel='y (m)')
plt.axis('equal')
plt.title('Distribution of the stations')
X = fiber_loc[0][900-250:1501-250]
Y = fiber_loc[1][900-250:1501-250]

In [ ]:
path = '518'
id = 889
tdms_files = glob.glob(datapath+'/*UTC_20220'+path+'_*.tdms')
tdms_files.sort()
data = np.array([])
for interval in range(6):
    dataa, dt, dx = das_read(tdms_files[interval])
    print(id+interval)
    dataa = dataa[300:2200,:]
    if interval == 0:
        data = dataa    
        continue
    data = np.concatenate((data,dataa),axis=1)
data = idas.das_preprocess(data)
data = idas.bandpass(data, dt, fl=1, fh=140)
data_521 = data[:,:]

path = '428'
id = 137
tdms_files = glob.glob(datapath+'/*UTC_20220'+path+'_*.tdms')
tdms_files.sort()
data = np.array([])
for interval in range(0,6):
    dataa, dt, dx = das_read(tdms_files[interval])
    print(id+interval)
    dataa = dataa[300:2200,:]
    if interval == 0:
        data = dataa    
        continue
    data = np.concatenate((data,dataa),axis=1)
data = idas.das_preprocess(data)
data = idas.bandpass(data, dt, fl=1, fh=140)
data_428 = data[:,:]

In [139]:
sources_428 = np.loadtxt('source_428_detailed.txt')
sources_518 = np.loadtxt('source_518_detailed.txt')
ft_time_428 = np.loadtxt('ft_time_428_detailed.txt')
ft_time_518 = np.loadtxt('ft_time_518_detailed.txt')
v_428 = np.loadtxt('v_428_detailed.txt')
v_518 = np.loadtxt('v_518_detailed.txt')

In [ ]:

from scipy import signal
sos = signal.butter(6, [1,2], 'bp', fs=1/dt, output='sos')
sg = np.abs(data_428[600:1200,10000:60000]).sum(axis=0)
sg = signal.detrend(sg)
sg = sg - np.mean(sg)
ev = signal.sosfiltfilt(sos, sg)
ft = signal.argrelextrema(ev, np.less)
plt.figure(figsize=(30,20)) 
plt.rcParams.update({'font.size': 35})
plt.subplot(211)
vlim = data_428[600:1200,10000+ft[0][10]:10000+ft[0][19]].std()
plt.imshow(data_428[600:1200,10000+ft[0][10]:10000+ft[0][19]], aspect='auto', cmap='seismic', vmin = -vlim, vmax=vlim,alpha=0.7)
plt.xticks(np.arange(0,1501,300),np.arange(0,5.1,1).astype(int))
plt.xlabel('Time (s)')
plt.ylabel('Channel')   
plt.subplot(212)
plt.plot(ev[10000+ft[0][10]:10000+ft[0][19]])
plt.plot(sg[10000+ft[0][10]:10000+ft[0][19]])
plt.xlim(0,ft[0][19]-ft[0][10])   
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')   
plt.xticks(np.arange(0,1501,300),np.arange(0,5.1,1).astype(int))


In [ ]:
from matplotlib.transforms import ScaledTranslation
fig, axs = plt.subplot_mosaic([['a)', 'b)','c)'], ['d)', 'e)','f)']],
                              layout='constrained', figsize=(60,30))   
plt.rcParams.update({'font.size': 60})  
for label, ax in axs.items():
    # Use ScaledTranslation to put the label
    # - at the top left corner (axes fraction (0, 1)),
    # - offset 20 pixels left and 7 pixels up (offset points (-20, +7)),
    # i.e. just outside the axes.
    ax.text(
        0.0, 1.0, label, transform=(
            ax.transAxes + ScaledTranslation(-1.15*6/1.8, -0.55*6/1.8, fig.dpi_scale_trans)),
        fontsize=90, va='bottom') 
colorbar_shrink = 0.8
plt.subplot(231)   
if True: 
    data_seg = data_428[600:1200,10000+ft[0][122]:10000+ft[0][122]+600]
    plt.imshow(data_seg/np.max(data_seg)*4, aspect='auto', cmap='seismic', vmin = -1, vmax=1)
    j = sources_428[123,0]
    i = sources_428[123,1]
    center = [j,i]  
    dist = ((X-center[0])**2+(Y-center[1])**2)**0.5
    plt.plot(dist*300/v_428[90]+210,np.arange(0,len(dist)),'black',linewidth=4)
    # plt.text(10, 40, 'V: '+str(v_arr[iv_max])+'m/s', fontsize=24)
    # plt.text(10, 80, 'location: '+str(xs_arr[ixs_max])+'   '+str(ys_arr[iys_max]), fontsize=24)
    plt.xlabel('Time (s)')
    plt.ylabel('Channel')
    colorbar = plt.colorbar(shrink=colorbar_shrink,boundaries=np.linspace(-1,1,100))
    colorbar.set_ticks(np.arange(-1,1.5,0.5)) 
    plt.xticks(np.arange(0,601,150),[0,0.5,1,1.5,2])
    plt.yticks(np.arange(0,601,100),np.arange(900,1501,100))
    plt.xlim(0,600)
    plt.ylim(600,0)
plt.subplot(233)
if True:
    data_521 = idas.das_preprocess(data_521)
    data_521 = idas.bandpass(data_521, dt, fl=1, fh=140)
    clim = data_521.std()*0.3
    plt.imshow(data_521[:,:], aspect='auto', cmap='seismic', vmin = -clim, vmax=clim)
    plt.xlabel('Time (min)')
    plt.ylim(1900,0)
    plt.text(5000,70,'A')
    plt.text(60000,70,'B')
    plt.ylabel('Channel')
    plt.vlines(5000,0,1900,colors='black')
    plt.vlines(60000,0,1900,colors='black')
    #plt.title('2022/05/18 22:44:00 - 2022/05/18 22:56:00')
    plt.xticks(np.arange(0,108010-18000*2,18000),np.arange(0,5,1))
    plt.yticks(np.arange(200,1900,500),np.arange(500,2200,500))
    plt.xlim(0,72010)
plt.subplot(2,3,2)
if True:
    source = np.zeros([22, 2])
    ft_time = np.zeros([22, 2])
    v = np.zeros([22, 1])

    # 定义每个中心点和源点
    centers_and_sources = [
        (225, 4.3, 240),
        (219, 8, 280),
        (209, 13, 330),
        (200, 20, 270),
        (196, 27, 280),
        (194, 35, 320),
        (191, 51, 320),
        (185, 72, 280),
        (182, 85, 320),
        (179, 100, 320),
        (172, 112, 320),
        (160, 120, 320),
        (155, 130, 280),
        (160, 140, 280)
        # ... 这里省略了剩余的源点定义，可以根据需要添加
    ]

    # 循环定义source和center
    for idx, (j, i, velocity) in enumerate(centers_and_sources):
        center = [j, i]
        source[idx] = center
        ft_time[idx] = [10780 + idx * 1000, 11400 + idx * 1000]  # 示例时间范围，根据实际情况调整
        v[idx] = velocity
        # 此处省略了数据处理和绘图的代码
    plt.imshow(data_428[:,:], aspect='auto', cmap='seismic', vmin = -clim, vmax=clim)
    plt.xlabel('Time (min)')
    plt.ylim(1900,0)
    plt.text(10000,70,'A')
    plt.text(60000,70,'B')
    plt.ylabel('Channel')
    plt.vlines(10000,0,1900,colors='black')
    plt.vlines(60000,0,1900,colors='black')
    #plt.title('2022/05/18 22:44:00 - 2022/05/18 22:56:00')
    plt.xlim(0,72010)
    plt.xticks(np.arange(0,108010-18000*2,18000),np.arange(0,5,1))
    plt.yticks(np.arange(200,1900,500),np.arange(500,2200,500))
plt.subplot(2,3,4) 
if True:
    xs = sources_428[91,0]
    ys = sources_428[91,1]  
    ag = 1.8849555921538759
    angle = np.ones(600)
    for i in range(600):
        angle[i] = compute_rotation_angle([xs-X[i],ys-Y[i]],[np.cos(ag),np.sin(ag)])
    # data_seg2 = np.copy(data_seg)
    # data_seg2[np.array(np.where(angle<np.pi/2)),:] = -10 * data_seg[np.array(np.where(angle<np.pi/2)),:]
    # data_seg2[np.array(np.where(angle>3*np.pi/2)),:] = -10 * data_seg[np.array(np.where(angle>3*np.pi/2)),:]

    plt.plot([xs-np.cos(ag+np.pi)*50,xs+np.cos(ag+np.pi)*50],[ys-np.sin(ag+np.pi)*50,ys+np.sin(ag+np.pi)*50],linewidth=4)
    plt.plot([xs-40*np.cos(ag+0.5*np.pi),xs+60*np.cos(ag+0.5*np.pi)],[ys-40*np.sin(ag+0.5*np.pi),ys+60*np.sin(ag+0.5*np.pi)],'r')
    plt.scatter(xs,ys,label='Source',s=200,alpha=0.8)
    #plt.plot(X[np.array([np.where((angle>np.pi*0.5)&(angle<1.5*np.pi))])][0],Y[np.array([np.where((angle>np.pi*0.5)&(angle<1.5*np.pi))])][0],'r')
    plt.plot(X,Y,'k',marker='^',markevery=100,markersize=25,label='Fiber')
    plt.plot(X[np.array(np.where(angle<np.pi/2))][0],Y[np.array(np.where(angle<np.pi/2))][0],'b')
    plt.plot(X[np.array(np.where(angle>3*np.pi/2))][0],Y[np.array(np.where(angle>3*np.pi/2))][0],'b')
    #plt.plot(X[np.array(np.where((angle>np.pi*0.5)&(angle<1.5*np.pi)))][0],Y[np.array(np.where((angle>np.pi*0.5)&(angle<1.5*np.pi)))[0]],'r')
    for i in range(6):
        if i == 3:
            plt.text(fiber_loc[0][900-250:1501-250:100][i],fiber_loc[1][900-250:1501-250:100][i]-14,str(900+i*100))
            continue
        plt.text(fiber_loc[0][900-250:1501-250:100][i],fiber_loc[1][900-250:1501-250:100][i],str(900+i*100))
    plt.xlim(50,350)
    plt.xlabel(xlabel='x (m)')
    plt.ylabel(ylabel='y (m)')
    plt.legend(loc='upper right',fontsize=45)   
    plt.axis('equal')
plt.subplot(2,3,6)
if True:
    source = sources_518
    plt.plot(fiber_loc[0][900-250:1500-250],fiber_loc[1][900-250:1500-250],'k',marker='^',markevery=100,markersize=25,label='Fiber') 
    plt.text(source[0][0],source[0][1],'A')
    plt.text(source[-1][0],source[-1][1],'B')
    plt.scatter(source[:,0],source[:,1],s=30,c='r')
    plt.scatter(source[0,0],source[0,1],s=60,c='r',label='Human footstep')
    for id in range(6):
        center = source[id]
        if id < 5:
            plt.plot([source[id][0],source[id+1][0]],[source[id][1],source[id+1][1]],c='r')
            # if id >=0:
            #     plt.arrow(source[id][0],source[id][1],(source[id+1][0]-source[id][0]),(source[id+1][1]-source[id][1]),head_width=5, head_length=5, fc='r', ec='r') 
    #plt.scatter(fiber_loc[0][900-235+300-15],fiber_loc[1][900-235+300-15])  
    for i in range(6):
        if i == 3:
            plt.text(fiber_loc[0][900-250:1501-250:100][i],fiber_loc[1][900-250:1501-250:100][i]-14,str(900+i*100))
            continue
        plt.text(fiber_loc[0][900-250:1501-250:100][i],fiber_loc[1][900-250:1501-250:100][i],str(900+i*100))
    plt.xlim(50,350)
    plt.xlabel(xlabel='x (m)')
    plt.ylabel(ylabel='y (m)')
    plt.legend(loc='upper right',fontsize=45)   
    plt.axis('equal')
plt.subplot(2,3,5)
if True:
    plt.plot(fiber_loc[0][900-250:1500-250],fiber_loc[1][900-250:1500-250],'k',marker='^',markevery=100,markersize=25,label='Fiber') 
    source = sources_428
    plt.text(source[0][0],source[0][1],'A')
    plt.text(source[-1][0],source[-1][1],'B')
    for id in range(7):
        center = source[id]
        if id < 6:
            plt.plot([source[id][0],source[id+1][0]],[source[id][1],source[id+1][1]],c='r')
            # if id >=0:
            #     plt.arrow(source[id][0],source[id][1],(source[id+1][0]-source[id][0]),(source[id+1][1]-source[id][1]),head_width=5, head_length=5, fc='r', ec='r')
    for i in range(6):
        if i == 3:
            plt.text(fiber_loc[0][900-250:1501-250:100][i],fiber_loc[1][900-250:1501-250:100][i]-14,str(900+i*100))
            continue
        plt.text(fiber_loc[0][900-250:1501-250:100][i],fiber_loc[1][900-250:1501-250:100][i],str(900+i*100))
    plt.scatter(source[:,0],source[:,1],s=30,c='r')
    plt.scatter(source[0,0],source[0,1],s=60,c='r',label='Human footstep')
    plt.xlim(50,350)
    plt.xlabel(xlabel='x (m)')
    plt.ylabel(ylabel='y (m)')
    plt.legend(loc='upper right',fontsize=45)   
    plt.axis('equal')
plt.savefig('b5.pdf',dpi=450)